In [47]:
import pandas as pd
import spacy
import re
import numpy as np
import datetime
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import csv
import urllib

In [16]:
# load events from excel
news_events = pd.read_excel('newsevents.xlsx', sheet_name='Sheet1')
news_events

,Event,Time
0,"Tornadoes strike Oklahoma, Texas, Arkansas, an...",2024-05-19
1,Boeing's Starliner capsule launches its first ...,2024-06-05
2,The Biden administration released revisions to...,2024-04-19
3,"In Planned Parenthood Arizona v. Mayes, the Ar...",2024-04-09


In [17]:
# do POS tagging and dependency parsing for every sentence to get subjec, object and predicate with spaCy
nlp = spacy.load('en_core_web_sm')

In [18]:
def is_work_token(s):
    return bool(re.fullmatch("[a-zA-Z0-9-]+", s) and s != "-" and not s.isnumeric())

In [19]:
# get the subject, object and predicate for every sentence
def get_subj_obj_predicate(sentence):
    doc = nlp(sentence)
    keywords = {'subject': set(), 'object': set(), 'predicate': set(), 'ner': set()}
    keywords_lemma = {'subject': set(), 'object': set(), 'predicate': set(), 'ner': set()}
    for idx, token in enumerate(doc):
        entry = ''
        if token.is_stop or (token.is_punct and token.text != '-'):
            continue

        if 'subj' in token.dep_ and is_work_token(token.text):
            entry = 'subject'
        elif 'obj' in token.dep_ and is_work_token(token.text):
            entry = 'object'
        elif ('ROOT' in token.dep_ or 'prd' in token.dep_) and is_work_token(token.text):
            entry = 'predicate'
        elif token.ent_type_ and is_work_token(token.text):
            entry = 'ner'

        if entry in keywords:
            keywords[entry].add(token.text)
            keywords_lemma[entry].add(token.lemma_)

        # if the work prev to the current one is compound, and the current one is a subject or object. Example: "Donald Trump"
        if idx != 0 and doc[idx-1].dep_ == 'compound' and ('subj' in token.dep_ or 'obj' in token.dep_):
            entry = 'subject' if 'subj' in token.dep_ else 'object'
            keywords[entry].add(doc[idx-1].text)
            keywords_lemma[entry].add(doc[idx-1].lemma_)

            if '-' in doc[idx-1].text:
                for word in doc[idx-1].text.split('-'):
                    if word.isalpha():
                        keywords[entry].add(word)
                        keywords_lemma[entry].add(word)

        # if the word has a hyphen, split it and add the words to the keywords. Example: COVID-19 (people may only use COVID)
        if '-' in token.text and entry:
            for word in token.text.split('-'):
                if word.isalpha():
                    keywords[entry].add(word)
                    keywords_lemma[entry].add(word)

        # if the compound word is mistakenly tagged, make it a ner entity. Example: alt-right could be mistakenly split into alt and right
        if token.text == '-':
            if idx != 0 and idx != len(doc)-1:
                tok = doc[idx-1].text + '-' + doc[idx+1].text
                keywords['ner'].add(tok)
                keywords_lemma['ner'].add(tok)
            
    return keywords, keywords_lemma

In [20]:
for idx, row in news_events.iterrows():
    try_time = 0
    keywords, keywords_lemma = get_subj_obj_predicate(row['Event'])
    news_events.at[idx, 'Subject'] = ', '.join(keywords['subject'])
    news_events.at[idx, 'Object'] = ', '.join(keywords['object'])
    news_events.at[idx, 'Predicate'] = ', '.join(keywords['predicate'])
    news_events.at[idx, 'NER'] = ', '.join(keywords['ner'])
    news_events.at[idx, 'Subject_lemma'] = ', '.join(keywords_lemma['subject'])
    news_events.at[idx, 'Object_lemma'] = ', '.join(keywords_lemma['object'])
    news_events.at[idx, 'Predicate_lemma'] = ', '.join(keywords_lemma['predicate'])  
    news_events.at[idx, 'NER_lemma'] = ', '.join(keywords_lemma['ner'])

In [21]:
news_events

,Event,Time,Subject,Object,Predicate,NER,Subject_lemma,Object_lemma,Predicate_lemma,NER_lemma
0,"Tornadoes strike Oklahoma, Texas, Arkansas, an...",2024-05-19,Tornadoes,"people, Oklahoma",strike,"Arkansas, Texas, Kentucky",tornado,"people, Oklahoma",strike,"Arkansas, Texas, Kentucky"
1,Boeing's Starliner capsule launches its first ...,2024-06-05,"Starliner, capsule","Space, delays, Force, Florida, space, Station,...",launches,"Canaveral, Space, Starliner, astronaut-crewed,...","Starliner, capsule","Space, Force, Florida, space, Station, flight,...",launch,"Canaveral, Space, Starliner, astronaut-crewed,..."
2,The Biden administration released revisions to...,2024-04-19,"Biden, administration","revisions, protections, rules, students, IX",released,"Biden, Title, IX","Biden, administration","revision, rule, IX, student, protection",release,"Biden, Title, IX"
3,"In Planned Parenthood Arizona v. Mayes, the Ar...",2024-04-09,"Court, Supreme","types, Parenthood, Mayes, abortions, law, Arizona",upholds,"Arizona, Supreme","Court, Supreme","type, Parenthood, Mayes, abortion, law, Arizona",uphold,"Arizona, Supreme"


## choose (obj OR ner) AND (subj OR prd) to web scraping for news
## timeframe: 1 month

In [22]:
def generate_keywords(event):
    obj_or_ner = f"({event['Object']} OR {event['NER']})"
    subj_or_prd = f"({event['Subject']} OR {event['Predicate']})"
    return f"{obj_or_ner} AND {subj_or_prd}"

In [24]:
for event in news_events.to_dict(orient='records'):
    keywords = generate_keywords(event)
    print(f"Generated Keywords for event '{event['Event']}': {keywords}")

Generated Keywords for event 'Tornadoes strike Oklahoma, Texas, Arkansas, and Kentucky, killing 21 people': (people, Oklahoma OR Arkansas, Texas, Kentucky) AND (Tornadoes OR strike)
Generated Keywords for event 'Boeing's Starliner capsule launches its first astronaut-crewed flight into space to the International Space Station after several delays at the Cape Canaveral Space Force Station in Florida.': (Space, delays, Force, Florida, space, Station, flight OR Canaveral, Space, Starliner, astronaut-crewed, Force, International, Cape, Boeing) AND (Starliner, capsule OR launches)
Generated Keywords for event 'The Biden administration released revisions to Title IX rules which give further protections for LGBTQ+ students as well as parenting and pregnant students.': (revisions, protections, rules, students, IX OR Biden, Title, IX) AND (Biden, administration OR released)
Generated Keywords for event 'In Planned Parenthood Arizona v. Mayes, the Arizona Supreme Court upholds an 1864 law that d

In [86]:
def get_date_range(event_date):
    if not isinstance(event_date, str):
        event_date = event_date.strftime('%Y-%m-%d')
    start_date = datetime.strptime(event_date, '%Y-%m-%d')
    end_date = start_date + timedelta(days=30)
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

def search_news(keywords, start_date, end_date):
    encoded_keywords = urllib.parse.quote(keywords)
    search_url = f"https://news.google.com/search?q={encoded_keywords}%20after:{start_date}%20before:{end_date}"
    articles = []

    while search_url:
        print(f"Searching for news with URL: {search_url}")
        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        for link in soup.find_all('a', class_='WwrzSb'):
            url = link.get('href')
            if url and url.startswith('./articles/'):
                full_url = f"https://news.google.com{url[1:]}"
                articles.append(full_url)

        next_button = soup.find('a', class_='nBDE1b G5eFlf')
        search_url = f"https://news.google.com{next_button['href'][1:]}" if next_button else None

    return articles

def collect_content(tag_list):
    return ' '.join([tag.get_text() for tag in tag_list])

def collect_content_USA(tag_list):
    return ' '.join([tag.get_text() for tag in tag_list])

def get_content(url):
    session = requests.Session()
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    print('getting content from url: ', url)

    if response.status_code != 200:
        print('Error with request')
        html = ''
    else:
        html = response.content
    soup = BeautifulSoup(html, 'html.parser')

    div_tag = soup.findAll('div', class_='article-body')
    div_tag2 = soup.findAll('div', class_='article-content')
    div_tag3 = soup.findAll('p', class_='gnt_ar_b_p')  # USA Today
    div_tag4 = soup.findAll('div', class_='article__text | body-copy | flow')  # The Hill
    div_tag5 = soup.findAll('div', class_='primary-cli cli cli-text')

    if div_tag:
        return collect_content(div_tag)
    elif div_tag2:
        return collect_content(div_tag2)
    elif div_tag3:
        return collect_content_USA(div_tag3)
    elif div_tag4:
        return collect_content(div_tag4)
    elif div_tag5:
        return collect_content(div_tag5)
    else:
        c_list = [v.text for v in soup.find_all('p') if len(v.text) > 0]
        words_to_bans = ['<', 'javascript']
        for word_to_ban in words_to_bans:
            c_list = list(filter(lambda x: word_to_ban not in x.lower(), c_list))
        c_list = [t for t in c_list if len(re.findall('[a-z]', t.lower())) / (len(t) + 1) < 0.8]
        content = ' '.join(c_list)
        content = content.replace('\n', ' ')
        content = re.sub('\s\s+', ' ', content)  # remove multiple spaces.
        return content

def get_article_details(url):
    try:
        article = Article(url)
        article.download()
        article.parse()

        # Check if the article text was parsed
        if not article.text:
            raise ValueError("No content parsed by newspaper3k")

        return article.publish_date, article.source_url, article.url, article.text

    except Exception as e:
        print(f"Error parsing article with newspaper3k: {e}")
        print("Attempting to manually extract content...")

        try:
            article_content = get_content(url)

            # Attempt to manually extract publish date
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            meta_date = soup.find('meta', {'property': 'article:published_time'})
            article_date = meta_date['content'] if meta_date else 'Unknown'

            return article_date, url, url, article_content

        except Exception as e:
            print(f"Failed to manually extract content from {url}: {e}")
            return None, None, url, ""

In [87]:
# Prepare CSV file
with open('newsarticles.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Event", "Event Date", "Article Date", "Article Publishers", "Article URL", "Article Content"])

    for event in news_events.to_dict(orient='records')[:1]:
        keywords = generate_keywords(event)
        start_date, end_date = get_date_range(event['Time'])
        print(f"Searching for news articles for event '{event['Event']}' between {start_date} and {end_date}")
        articles = search_news(keywords, start_date, end_date)
        for article_url in articles:
            try:
                article_date, article_publishers, article_url, article_content = get_article_details(article_url)
                writer.writerow([event['Event'], event['Time'], article_date, article_publishers, article_url, article_content])
            except Exception as e:
                print(f"Failed to fetch article from {article_url}: {e}")

Searching for news articles for event 'Tornadoes strike Oklahoma, Texas, Arkansas, and Kentucky, killing 21 people' between 2024-05-19 and 2024-06-18
Searching for news with URL: https://news.google.com/search?q=%28people%2C%20Oklahoma%20OR%20Arkansas%2C%20Texas%2C%20Kentucky%29%20AND%20%28Tornadoes%20OR%20strike%29%20after:2024-05-19%20before:2024-06-18
Error parsing article with newspaper3k: No content parsed by newspaper3k
Attempting to manually extract content...
getting content from url:  https://news.google.com/articles/CBMibmh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL3Rvcm5hZG9lcy13ZWF0aGVyLXRleGFzLW9rbGFob21hLWFya2Fuc2FzLWtlbnR1Y2t5LTkxYjBmZjVjMDZmOWJkYzhiOGNhNjhmMzZhZmJhZmEy0gEA?hl=en-US&gl=US&ceid=US%3Aen
Error parsing article with newspaper3k: No content parsed by newspaper3k
Attempting to manually extract content...
getting content from url:  https://news.google.com/articles/CBMiSmh0dHBzOi8vd3d3Lm5wci5vcmcvMjAyNC8wNS8yNi9nLXMxLTEwOTIvdG9ybmFkb2VzLXRleGFzLW9rbGFob21hLWFya2Fuc2Fz0gEA

In [65]:
search_url = 'https://news.google.com/search?q=%28people%2C%20Oklahoma%20OR%20Arkansas%2C%20Texas%2C%20Kentucky%29%20AND%20%28Tornadoes%20OR%20strike%29%20after:2024-05-19%20before:2024-06-18'

In [66]:
articles = []

while search_url:
    print(f"Searching for news with URL: {search_url}")
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for link in soup.find_all('a', class_='WwrzSb'):
        url = link.get('href')
        if url and url.startswith('./articles/'):
            full_url = f"https://news.google.com{url[1:]}"
            articles.append(full_url)

    next_button = soup.find('a', class_='nBDE1b G5eFlf')
    search_url = f"https://news.google.com{next_button['href'][1:]}" if next_button else None


Searching for news with URL: https://news.google.com/search?q=%28people%2C%20Oklahoma%20OR%20Arkansas%2C%20Texas%2C%20Kentucky%29%20AND%20%28Tornadoes%20OR%20strike%29%20after:2024-05-19%20before:2024-06-18


In [81]:
article = Article('https://news.google.com/articles/CBMibmh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL3Rvcm5hZG9lcy13ZWF0aGVyLXRleGFzLW9rbGFob21hLWFya2Fuc2FzLWtlbnR1Y2t5LTkxYjBmZjVjMDZmOWJkYzhiOGNhNjhmMzZhZmJhZmEy0gEA?hl=en-US&gl=US&ceid=US%3Aen')
article.download()
article.parse()
article.publish_date, article.source_url, article.url, article.text

(None,
 'https://news.google.com',
 'https://news.google.com/articles/CBMibmh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL3Rvcm5hZG9lcy13ZWF0aGVyLXRleGFzLW9rbGFob21hLWFya2Fuc2FzLWtlbnR1Y2t5LTkxYjBmZjVjMDZmOWJkYzhiOGNhNjhmMzZhZmJhZmEy0gEA?hl=en-US&gl=US&ceid=US%3Aen',
 '')

In [68]:
len(articles)

81

try serpi

In [91]:
api = '66ac47018bbaa4304deef8e4'

In [94]:
def search_news(keywords, start_date, end_date, num_results=10, num_pages=5):
    articles = []
    for page in range(0, num_pages * num_results, num_results):
        payload = {
            'api_key': '66ac47018bbaa4304deef8e4',
            'q': keywords,
            'gl': 'us',
            'hl': 'en_us',
            'from': start_date,
            'to': end_date,
            'num': num_results,
            'start': page
        }
        resp = requests.get('https://api.serpdog.io/news', params=payload)
        if resp.status_code == 200:
            articles.extend(resp.json().get('news_results', []))
        else:
            print(f"Error fetching page {page // num_results + 1}: {resp.status_code}")
            break
    return articles

In [96]:
with open('news_results1.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the headers
    csv_writer.writerow(["Event", "Event Date", "title", "snippet", "source", "lastUpdated", "url"])

    for event in news_events.to_dict(orient='records')[:1]:
        keywords = generate_keywords(event)
        start_date, end_date = get_date_range(event['Time'])
        print(f"Searching for news articles for event '{event['Event']}' between {start_date} and {end_date}")
        news_results = search_news(keywords, start_date, end_date)
        
        # Write the data
        for result in news_results:
            csv_writer.writerow([event['Event'], event['Time'], result["title"], result["snippet"], result["source"], result["lastUpdated"], result["url"]])
        
print('Done writing to CSV file.')

Searching for news articles for event 'Tornadoes strike Oklahoma, Texas, Arkansas, and Kentucky, killing 21 people' between 2024-05-19 and 2024-06-18
Done writing to CSV file.


In [97]:
with open('news_results2.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the headers
    csv_writer.writerow(["Event", "Event Date", "title", "snippet", "source", "lastUpdated", "url"])

    for event in news_events.to_dict(orient='records')[1:2]:
        keywords = generate_keywords(event)
        start_date, end_date = get_date_range(event['Time'])
        print(f"Searching for news articles for event '{event['Event']}' between {start_date} and {end_date}")
        news_results = search_news(keywords, start_date, end_date)
        
        # Write the data
        for result in news_results:
            csv_writer.writerow([event['Event'], event['Time'], result["title"], result["snippet"], result["source"], result["lastUpdated"], result["url"]])
        
print('Done writing to CSV file.')

Searching for news articles for event 'Boeing's Starliner capsule launches its first astronaut-crewed flight into space to the International Space Station after several delays at the Cape Canaveral Space Force Station in Florida.' between 2024-06-05 and 2024-07-05
Done writing to CSV file.


In [98]:
with open('news_results3.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the headers
    csv_writer.writerow(["Event", "Event Date", "title", "snippet", "source", "lastUpdated", "url"])

    for event in news_events.to_dict(orient='records')[2:3]:
        keywords = generate_keywords(event)
        start_date, end_date = get_date_range(event['Time'])
        print(f"Searching for news articles for event '{event['Event']}' between {start_date} and {end_date}")
        news_results = search_news(keywords, start_date, end_date)
        
        # Write the data
        for result in news_results:
            csv_writer.writerow([event['Event'], event['Time'], result["title"], result["snippet"], result["source"], result["lastUpdated"], result["url"]])
        
print('Done writing to CSV file.')

Searching for news articles for event 'The Biden administration released revisions to Title IX rules which give further protections for LGBTQ+ students as well as parenting and pregnant students.' between 2024-04-19 and 2024-05-19
Done writing to CSV file.


In [99]:
with open('news_results4.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the headers
    csv_writer.writerow(["Event", "Event Date", "title", "snippet", "source", "lastUpdated", "url"])

    for event in news_events.to_dict(orient='records')[3:4]:
        keywords = generate_keywords(event)
        start_date, end_date = get_date_range(event['Time'])
        print(f"Searching for news articles for event '{event['Event']}' between {start_date} and {end_date}")
        news_results = search_news(keywords, start_date, end_date)
        
        # Write the data
        for result in news_results:
            csv_writer.writerow([event['Event'], event['Time'], result["title"], result["snippet"], result["source"], result["lastUpdated"], result["url"]])
        
print('Done writing to CSV file.')

Searching for news articles for event 'In Planned Parenthood Arizona v. Mayes, the Arizona Supreme Court upholds an 1864 law that disallows most types of abortions' between 2024-04-09 and 2024-05-09
Done writing to CSV file.


Content scraping

In [100]:
# Function to scrape news content from the given URL
def get_content(url):
    session = requests.Session()
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    print('Getting content from URL:', url)

    if response.status_code != 200:
        print('Error with request')
        html = ''
    else:
        html = response.content

    soup = BeautifulSoup(html, 'html.parser')

    div_tag = soup.find_all('div', class_='article-body')
    div_tag2 = soup.find_all('div', class_='article-content')
    div_tag3 = soup.find_all('p', class_='gnt_ar_b_p')  # USA Today
    div_tag4 = soup.find_all('div', class_='article__text | body-copy | flow')  # The Hill
    div_tag5 = soup.find_all('div', class_='primary-cli cli cli-text')

    if div_tag:
        return collect_content(div_tag)
    elif div_tag2:
        return collect_content(div_tag2)
    elif div_tag3:
        return collect_content_USA(div_tag3)
    elif div_tag4:
        return collect_content(div_tag4)
    elif div_tag5:
        return collect_content(div_tag5)
    else:
        c_list = [v.text for v in soup.find_all('p') if len(v.text) > 0]
        words_to_bans = ['<', 'javascript']
        for word_to_ban in words_to_bans:
            c_list = list(filter(lambda x: word_to_ban not in x.lower(), c_list))
        c_list = [t for t in c_list if len(re.findall('[a-z]', t.lower())) / (len(t) + 1) < 0.8]
        content = ' '.join(c_list)
        content = content.replace('\n', ' ')
        content = re.sub('\s\s+', ' ', content)  # remove multiple spaces.
        return content

def collect_content(tag_list):
    return ' '.join([tag.get_text() for tag in tag_list])

def collect_content_USA(tag_list):
    return ' '.join([tag.get_text() for tag in tag_list])


In [101]:
# Read the existing CSV to get the list of URLs and other information
event1 = pd.read_csv('news_results1.csv')

# Add a new column for the content
event1['content'] = ''

# Loop through each row to get content for each URL
for index, row in event1.iterrows():
    url = row['url']
    content = get_content(url)
    event1.at[index, 'content'] = content


Getting content from URL: https://apnews.com/article/tornadoes-weather-texas-oklahoma-arkansas-kentucky-91b0ff5c06f9bdc8b8ca68f36afbafa2
Getting content from URL: https://www.npr.org/2024/05/26/g-s1-1092/tornadoes-texas-oklahoma-arkansas
Getting content from URL: https://www.cnn.com/weather/live-news/oklahoma-texas-arkansas-tornado-storms-05-27-24-intl-hnk/index.html
Getting content from URL: https://www.wgbh.org/news/2024-05-26/more-than-20-are-dead-after-tornadoes-rip-through-parts-of-texas-kentucky-arkansas
Getting content from URL: https://www.nytimes.com/2024/05/26/us/tornadoes-storms-texas-oklahoma.html
Error with request
Getting content from URL: https://weather.com/news/weather/news/2024-05-25-deadly-tornado-texas-oklahoma-arkansas-storms
Getting content from URL: https://www.cbsnews.com/news/storms-kill-at-least-19-texas-kentucky-arkansas-oklahoma/
Getting content from URL: https://www.reuters.com/world/us/tornadoes-hit-texas-arkansas-oklahoma-killing-least-2-texas-nyt-reports

In [107]:
# drop empty content
event1.drop(event1[event1['content'] == ''].index, inplace=True)

In [108]:
event2 = pd.read_csv('news_results2.csv')
event3 = pd.read_csv('news_results3.csv')
event4 = pd.read_csv('news_results4.csv')

In [109]:
def add_content_to_event(event):
    # Add a new column for the content
    event['content'] = ''

    # Loop through each row to get content for each URL
    for index, row in event.iterrows():
        url = row['url']
        content = get_content(url)
        event.at[index, 'content'] = content
    return event

In [110]:
event2 = add_content_to_event(event2)
event3 = add_content_to_event(event3)
event4 = add_content_to_event(event4)

Getting content from URL: https://www.livescience.com/space/space-exploration/boeing-starliner-astronauts-remain-stuck-on-international-space-station-with-no-set-return-date-nasa-announces
Getting content from URL: https://abcnews.go.com/US/boeings-starliner-set-1st-astronaut-crewed-flight-after/story?id=110855655
Getting content from URL: https://www.nbcnews.com/science/space/live-blog/live-updates-boeing-starliner-launch-nasa-astronauts-space-rcna155185
Getting content from URL: https://www.npr.org/2024/06/05/nx-s1-4956024/boeing-starliner-launch-nasa-international-space-station
Getting content from URL: https://www.latimes.com/business/story/2024-07-25/nasa-boeing-starliner-international-space-station-thrusters-helium-leak
Getting content from URL: https://www.aljazeera.com/news/2024/6/27/why-are-two-boeing-astronauts-stuck-in-space
Getting content from URL: https://www.mysanantonio.com/news/local/article/boeing-starliner-19581736.php
Getting content from URL: https://www.reuters.co

In [114]:
# combine and drop empty content
events = pd.concat([event1, event2, event3, event4])
events.drop(events[events['content'] == ''].index, inplace=True)

In [115]:
# Save the final data to a new CSV file
events.to_csv('news_results_content.csv', index=False)